In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
model_name = "BAAI/bge-m3"
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=0,
    chunk_overlap=0
)

loader = TextLoader("./docs/travel.txt", encoding="utf-8")
documents = loader.load_and_split(text_splitter=text_splitter)

Created a chunk of size 52, which is longer than the specified 0
Created a chunk of size 46, which is longer than the specified 0
Created a chunk of size 43, which is longer than the specified 0
Created a chunk of size 47, which is longer than the specified 0
Created a chunk of size 41, which is longer than the specified 0
Created a chunk of size 41, which is longer than the specified 0
Created a chunk of size 47, which is longer than the specified 0
Created a chunk of size 38, which is longer than the specified 0
Created a chunk of size 40, which is longer than the specified 0
Created a chunk of size 41, which is longer than the specified 0
Created a chunk of size 38, which is longer than the specified 0
Created a chunk of size 46, which is longer than the specified 0
Created a chunk of size 36, which is longer than the specified 0
Created a chunk of size 35, which is longer than the specified 0
Created a chunk of size 45, which is longer than the specified 0
Created a chunk of size 3

In [5]:
from langchain_chroma import Chroma
db = Chroma.from_documents(documents, hf_embeddings)

In [6]:
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 20, "lambda_mult": 0.5}
)

In [7]:
from langchain_core.prompts import ChatPromptTemplate
message = """
Answer this question using the provided context only.
아래 제공된 내용만 참고하여 사용자의 질문에 답변해줘. 모르면 솔직하게 그냥 모른다고 답해.

Context:
{context}

User Input:
{input}
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("human", message)
])

In [8]:
from langchain import hub
# https://smith.langchain.com/hub/langchain-ai/retrieval-qa-chat
prompt_template = hub.pull("langchain-ai/retrieval-qa-chat")
prompt_template

c:\Python311\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMes

In [9]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [10]:
from langchain_core.runnables import RunnablePassthrough
rag_chain = {
    "context": retriever,
    "input": RunnablePassthrough()
 } | prompt_template | llm | parser

In [11]:
rag_chain.invoke("이색적인 숙소에는 어떤 곳이 있을까?")

'이색적인 숙소로는 트리하우스, 동굴 호텔, 빙하 호텔 등이 있습니다. 이러한 숙소들은 독특한 숙박 경험을 제공하여 여행의 즐거움을 더해줍니다.'

In [12]:
rag_chain.invoke("우주의 역사에 대해서 알려줘")

'죄송하지만, 제공된 문서의 내용은 여행 관련 정보에 한정되어 있어 우주의 역사에 대한 정보는 포함되어 있지 않습니다. 다른 질문이나 여행 관련 정보가 필요하시면 도와드리겠습니다.'